# LlamaIndex + Strata Integration

This tutorial demonstrates how to build AI agents using LlamaIndex with Klavis Strata MCP servers for enhanced functionality.


## Prerequisites

Before we begin, you'll need:

- **OpenAI API key** - Get at [openai.com](https://openai.com/)
- **Klavis API key** - Get at [klavis.ai](https://klavis.ai/)


In [1]:
# Install the required packages
%pip install -q klavis python-dotenv llama-index-llms-openai llama-index-tools-mcp llama-index-core



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:

import os
import webbrowser
from klavis import Klavis
from klavis.types import McpServerName
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.tools.mcp import BasicMCPClient, aget_tools_from_mcp_url

# Set environment variables
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Replace with your actual OpenAI API key
os.environ["KLAVIS_API_KEY"] = "YOUR_KLAVIS_API_KEY"  # Replace with your actual Klavis API key

## Step 1: Create Strata MCP Server

Create a unified MCP server that combines multiple services (Gmail and YouTube) for enhanced agent capabilities.


In [9]:
klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

# Create a Strata MCP server with Gmail and YouTube integrations
response = klavis_client.mcp_server.create_strata_server(
    user_id="1234",
    servers=[McpServerName.GMAIL, McpServerName.YOUTUBE],
)

print(f"🚀 Strata MCP server created successfully!")

# Handle OAuth authorization if needed
if response.oauth_urls:
    for server_name, oauth_url in response.oauth_urls.items():
        webbrowser.open(oauth_url)
        print(f"🔐 Opening OAuth authorization for {server_name}")
        input(f"Press Enter after completing {server_name} OAuth authorization...")


🚀 Strata MCP server created successfully!
🔐 Opening OAuth authorization for Gmail


## Step 2: Create LlamaIndex Agent with MCP Tools

Set up the LlamaIndex FunctionAgent with tools from the Strata MCP server.


In [15]:
# Get all available tools from Strata
tools = await aget_tools_from_mcp_url(
    response.strata_server_url, 
    client=BasicMCPClient(response.strata_server_url)
)

# Setup LLM
llm = OpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

# Create LlamaIndex agent with MCP tools
agent = FunctionAgent(
    name="my_first_agent",
    description="Agent using MCP-based tools",
    tools=tools,
    llm=llm,
    system_prompt="You are an AI assistant that uses MCP tools.",
)

print("🤖 LlamaIndex agent created with MCP tools!")

🤖 LlamaIndex agent created with MCP tools!


## Step 3: Run the Agent

Use the agent to summarize a YouTube video and send the summary via email.


In [16]:
# Configure your preferences
my_email = "your-email@example.com"  # Replace with your email
youtube_video_url = "https://youtu.be/yebNIHKAC4A?si=1Rz_ZsiVRz0YfOR7"  # Replace with your favorite video

# Invoke the agent
response = await agent.run(
    f"summarize this video - {youtube_video_url} and mail this summary to my email {my_email}"
)

print("✅ Result:")
print(response)

✅ Result:
### Video Summary
The video titled **“Golden” Official Lyric Video | KPop Demon Hunters | Sony Animation** features the official lyric video for the song "Golden" from the KPop Demon Hunters soundtrack. The KPop Demon Hunters series is available on Netflix and follows KPop superstars Rumi, Mira, and Zoey, who lead double lives as demon hunters protecting their fans from supernatural threats. The video showcases vibrant animations and catchy lyrics, appealing to fans of K-Pop and animation alike.

**Key Details:**
- **Published At:** June 23, 2025
- **Channel:** Sony Pictures Animation
- **Duration:** 3 minutes and 19 seconds
- **View Count:** 372,127,057
- **Like Count:** 2,861,033
- **Comment Count:** 87,798
- **Watch Here:** [Golden - KPop Demon Hunters](https://www.youtube.com/watch?v=yebNIHKAC4A)

### Email Sent
The summary has been successfully sent to your email at **your-email@example.com**.


## Summary

🎉 Congratulations! You've successfully created a LlamaIndex agent that can:

1. **Summarize YouTube videos** using the YouTube MCP server
2. **Send emails** using the Gmail MCP server
3. **Coordinate multiple services** through Klavis Strata MCP integration

This demonstrates the power of combining LlamaIndex's FunctionAgent with Klavis MCP servers for building sophisticated AI workflows that can interact with multiple external services seamlessly.
